# PyZX interoperability: phases and T-gates.

*(This notebook is still being written. Take it as general idea, not final version.)*

This notebook offers an example of algorithmic lattice surgery with an emphasis on applying phases in the original PyZX graph to the lattice surgery results produced by the algorithm, and using that information to determine with of the blocks in the final result are T-gates.

In [ ]:
import os
import sys
import pyzx as zx
from pathlib import Path
from fractions import Fraction
from pyzx.utils import EdgeType
zx.settings.colors = zx.rgb_colors
%matplotlib widget

repository_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
output_folder_path = f"{repository_root}/outputs/txt"
if repository_root not in sys.path:
    sys.path.insert(0, repository_root)

from scripts.runner import runner
from utils.interop_pyzx import get_simple_graph_from_pyzx


## Input: Preparing a PyZX graph

Let's start by preparing a PyZX graph. We'll use a graph that is large enough to allow a variety of spiders with and without phases but also sufficiently short as to not have to wait too long for the foundational algorithmic result to be ready for manipulation.

In [ ]:


circuit_name = "pyzx_example_t_gates"

c = zx.Circuit(3)
c.add_gate("ZPhase", 0, phase=Fraction(1,2))
c.add_gate("CNOT", 0, 2)
c.add_gate("CNOT", 1, 2)
c.add_gate("CNOT", 0, 2)
c.add_gate("ZPhase", 1, phase=Fraction(1,1))
c.add_gate("ZPhase", 2, phase=Fraction(1,4))

pyzx_graph = c.to_graph()
pyzx_graph.add_edge(edge_pair=(8,5),  edgetype=EdgeType.SIMPLE)
pyzx_graph.add_to_phase(vertex=7, phase=Fraction(1/2))
pyzx_graph.add_to_phase(vertex=10, phase=Fraction(1/4))

zx.draw(pyzx_graph, labels=True)

## Process: Running the algorithm

To feed the graph into the algorithm, we first need to convert it into a `simple_graph`, i.e., a simple dictionary of nodes and edges. The conversion removes information that is not needed for foundational operations, i.e., converting the graph into a 3D version of itself. We will recover that information later in the process.

In [ ]:
simple_graph = get_simple_graph_from_pyzx(pyzx_graph)
print(simple_graph)

The next step is to feed the `simple_graph` to the algorithm and let it work its magic (it's not magic, but let's pretend it is).

If/when the algorithm is successful, it will produce:
- an interactive visualisation displayed on screen,
- an animation of the process saved to `./outputs/media/`,
- node-by-node/edge-by-edge text results saved to `./outputs/txt/`,
- and related objects for programmatic use.

It is good practice to examine the results printed to screen, too. In short, the algorithm visits all spiders in a ZX-graph and exchanges them to a 3D version of themselves, and processes edges by also encoding them as a special kind of 3D edge. The printouts on screen detail the order of operations.

**Note!** This notebook limits the number of attempts by the algorithm to a single attempt using the optional `max_attempts=1` parameter. You may need to run the following block a few times to get a successful result. The standard approach is to give the algorithm 10 attempts. The number of attempts is limited here for readability. 

In [ ]:
# Define weights for the value function to to choose best of several valid paths per each edge based on: (length of path, number of beams broken by path)
# A negative value for length of path favours short paths.
# A negative value for number of beams broken by path favours placements that do not block potential open faces requiring connections.
VALUE_FUNCTION_HYPERPARAMS = (-1, -0.5)

# Define a desired length of beams
# The longer the beams, the more space between faces. This feature needs improvement but, in theory, increases the odds of success.
# Needs to be combined with an equal or larger `MAX_PATHFINDER_SEARCH_SPACE`
LENGTH_OF_BEAMS = 3

# Define the maximum size of the search space (distance between source and target node for each pathfinding iteration)
# Larger values will produce a larger number of paths, and longer paths. This increases the odds of a successful result, with a significant impact on runtime.
MAX_PATHFINDER_SEARCH_SPACE = 3

kwargs = {
    "weights": VALUE_FUNCTION_HYPERPARAMS,
    "length_of_beams": LENGTH_OF_BEAMS,
    "max_search_space": MAX_PATHFINDER_SEARCH_SPACE,
}

simple_graph_after_use, edge_paths, lattice_nodes, lattice_edges = runner(
    simple_graph, circuit_name, strip_boundaries=False, hide_boundaries=False, max_attempts=1, visualise="final", **kwargs
)

## Output: Transferring phases and identifying T-gates in outputs

It is always helpful to visualise the relation between the original PyZX graph and results by printing the graph alongside a summary of how the algorithm rendered each edge.
- Short edges containing a three step combination of block-pipe-block or node-edge-node are edges that the algorithm cleared easily by simply placing the next spider.
- Long edges containing more than three items correspond to edges the algorithm had to break into several segments to clear in the 3D space.

In [ ]:
from utils.utils_zx_graphs import get_zx_type_from_kind
zx.draw(pyzx_graph, labels=True)

if edge_paths:
    for key, edge in edge_paths.items():
        block_by_block = []
        nodes_and_edges = []
        for node in edge["path_nodes"]:
            block_by_block.append(node[1])
            nodes_and_edges.append(get_zx_type_from_kind(node[1]))
        print(f"{key}: {"-".join(block_by_block)} ({" - ".join(nodes_and_edges)})")

You can also print the final `lattice_nodes` and `lattice_edges` objects for inspection. These would be the objects one would use to transfer the space-time diagram to a different system. They offer a block-by-block 3D representation of the original ZX-graph.

As you can observe from these objects, some of the 3D blocks in `lattice_nodes` have the same IDs as the spiders in the original ZX graph. These blocks are the direct representation of the original spider sharing the ID. Blocks with IDs different to those in the original ZX graph are additional intermediary nodes that the algorithm had to place to be able to clear some of the edges in the 3D space.

The same is true of the 3D blocks in `lattice_edges`. The edges that have the same (source, target) IDs are a direct representation of the corresponding edges in the original ZX graph. Additional edges may or may not be necessary depending on the graph, but essentially mean the algorithm had to break one of the original edges into several subsegments in order to be able to clear the path in 3D.

In [ ]:
if lattice_nodes:
    for key, node in lattice_nodes.items():
        print(f"Node ID {key}: {node}.")

if lattice_edges:
    for key, edge in lattice_edges.items():
        print(f"Edge ID {key}: {edge[0]} (corresponding edge in ZX-graph: {edge[1]}).")

As ellaborated [here](https://arxiv.org/pdf/1902.03178) and [here](https://arxiv.org/pdf/1903.10477), non-Clifford phases are those which are not multiples of π/2.

It is therefore possible to identify T-gates in the final space-time diagram in two steps:
- Transfer the phases of the original spiders to their corresponding pair in `lattice_nodes`, 
- Use phases to to identify T-gates.

In [ ]:
map_of_phases = pyzx_graph.phases()
lattice_nodes_extended = {}

if lattice_nodes:
    for id, node in lattice_nodes.items():
        position, kind = node
        phase = map_of_phases[id] if id in map_of_phases.keys() else 0
        gate_type = "T" if (isinstance(phase, Fraction) and phase % Fraction(1/2) != Fraction(0/1)) else "Clifford"
        lattice_nodes_extended[id] = (position, kind, phase, gate_type)

Finally, we can review the result by printing it to screen and/or appending it to previously saved text outputs.

In [ ]:
for id, node in lattice_nodes_extended.items():
    print(f"{id}: {node}")

lines = []
if lattice_nodes_extended and lattice_edges:
    lines.append(
        "\n__________________________\nLATTICE SURGERY EXTENDED (Graph + Phases)\n"
    )
    for key, node in lattice_nodes_extended.items():
        lines.append(f"Node ID: {key}. Info: {node}\n")
    for key, edge_info in lattice_edges.items():
         lines.append(f"Edge ID: {key}. Kind: {edge_info[0]}. Original edge in ZX graph: {edge_info[1]} \n")

    Path(output_folder_path).mkdir(parents=True, exist_ok=True)
    with open(f"{output_folder_path}/{circuit_name}.txt", "a") as f:
        f.writelines(lines)
        f.close()

    print(f"\nResult saved to: <...>/{circuit_name}.txt")